In [1]:
import requests as req
import pandas as pd
import json
from PIL import Image
from io import BytesIO
import random
from tqdm import tqdm
from oracle_test import OracleTools
import my_utils as mu
import time
import multiprocessing as mp

In [95]:
### ERROR가 아닌 타입의 로그가 있는 경우 제대로 분리해내지 못하는 상태임. << 수정 필요
def openLogAsJsonList(path: str, log_level: str='ERROR'):
    log_level = log_level.upper()
    tmp = open(path, 'rt', encoding='UTF8').read()
    tmp = tmp.replace('\n', '')
    tmp = tmp.split(f"{log_level}:root:")
    tmp.remove('')
    tmp = list(filter(lambda x: x[0] == '{', tmp))
    return [json.loads(a.replace('\'', '"')) for a in tmp]

In [7]:
sample = mu.getSampleData('gold', 1, 1, 'RGAPI-5db2eccc-c2fd-4161-82cc-ed33eb124784')
mat = sample.iloc[0]['matches']['info']
tim = sample.iloc[0]['timeline']['info']

get SummonerName.....


100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


total player:  1
get puuid......


100%|██████████| 1/1 [00:00<00:00,  3.52it/s]


passed player:  0
get match_id...


100%|██████████| 1/1 [00:00<00:00,  3.60it/s]


get matches & timeline.....


100%|██████████| 1/1 [00:00<00:00,  1.29it/s]

complete!


In [84]:
rawdatas = mu.oracle_totalExecute('SELECT * FROM RAWDATA')

oracle open!
oracle close!


In [229]:
rawdatas.sort_values(['GAME_ID', 'PARTICIPANT_NUMBER'], ascending=[False, True], ignore_index=True, inplace=True)

In [133]:
sample_raw = rawdatas.head(10)

### 타임라인에 필요한 데이터:
* 챔피언 스킬 빌드
* 챔피언 시작 아이템
* 챔피언 아이템 빌드

#### 챔피언 스킬 빌드

In [67]:
skill1 = mu.eventExtractor(sample.iloc[0], 'SKILL_LEVEL_UP', 1)

In [68]:
build1 = [a['skillSlot'] for a in skill1]
build1

[1, 3, 2, 3, 3, 4, 3, 1, 3, 1, 4, 1, 1, 2, 2]

#### 챔피언 시작 아이템

In [70]:
with open('D:\\7_LOL Project\\dragontail-13.8.1\\13.8.1\\data\\ko_KR\\item.json', 'rt', encoding='utf8') as f:
    items = json.loads(f.read())

In [8]:
trinket = [3340,3363,3364]

In [57]:
def getStartItem(rawdata_series: pd.Series, participant_number: int):
    parts = [a['teamPosition'] for a in rawdata_series['matches']['info']['participants']]

    item_pur = mu.eventExtractor(rawdata_series, 'ITEM_PURCHASED', participant_number)
    purchased = [a['itemId'] for a in filter(lambda x: x['timestamp']<=120000, item_pur)]

    item_undo = mu.eventExtractor(rawdata_series, 'ITEM_UNDO', participant_number)
    undo = [a['beforeId'] for a in filter(lambda x: x['timestamp']<=120000, item_undo)]
    for remo in undo:
        purchased.remove(remo)

    return {'teamPosition': parts[participant_number-1], 'purchasedItems': purchased}

In [58]:
def itemId2Name(item_dict: dict):
    with open('D:\\7_LOL Project\\dragontail-13.8.1\\13.8.1\\data\\ko_KR\\item.json', 'rt', encoding='utf8') as f:
        items = json.loads(f.read())
    item_dict['purchasedItems'] = [items['data'][str(a)]['name'] for a in item_dict['purchasedItems']]
    return item_dict

In [280]:
ids = [getStartItem(sample.iloc[0], i) for i in range(1, 11)]
ids[0]

{'teamPosition': 'TOP', 'purchasedItems': [1054, 3340, 2003]}

In [281]:
t = [itemId2Name(getStartItem(sample.iloc[0], i)) for i in range(1, 11)]
t[0]

{'teamPosition': 'TOP', 'purchasedItems': ['도란의 방패', '투명 와드', '체력 물약']}

#### 아이템 빌드

#### 밴률

In [272]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '"True"').replace('False', '"False"'))

In [273]:
ban_list = []
for loc in range(0, len(rawdatas), 10):
    matches = rawdataLoader(rawdatas.iloc[loc]['MATCHES'])
    bans = [a['championId'] for a in matches['game']['bans']]
    ban_list.append({'game_id':rawdatas.iloc[loc]['GAME_ID'], 'bans':bans})

In [252]:
champs = mu.oracle_totalExecute('SELECT CHAMP_ID, CHAMP_NAME FROM CHAMPDATA')

oracle open!
oracle close!


In [ ]:
champs.sort_values('CHAMP_ID', ignore_index=True, inplace=True)
champs

In [261]:
champs['BAN_TIME'] = champs.apply(lambda x: 0, axis=1)

In [275]:
champ_id_list = [a for a in champs['CHAMP_ID']]

In [276]:
ban_time = {a:0 for a in champ_id_list}
for lst in ban_list:
    for ban in lst['bans']:
        if ban == -1: continue
        ban_time[ban] += 1
ban_time

In [278]:
ban_rate = {a:round((ban_time[a] / len(ban_list) * 100), 2) for a in ban_time}
ban_rate